# Leading Living Standards Index
The objective of the Global Leading Living Standards Index is to compile a list of the best countries to reside in. To accomplish this, an analysis will be conducted using a set of 48 distinct metrics. These metrics include various indices, surveys, and annual data points which cover different living standard contributors.

## 1) Converting raw data to scores
Data is collected in its raw format using various web scraping scripts, utilising Python's Beautiful Soup and Selenium libraries. Once the data is collected, it needs to be standardised to convert raw scores into a value between 0 and 1. This is achieved by using min-max standardisation.


## 2) Calculation of subindex scores
Each indicator is assigned to one of the four subindices ('Economic health and stability,' 'Quality of Life,' 'Cost of Living,' 'Growth and Development'). Within each subindex, all indicators are given a relative subindex weighting that represents the relevance and importance of each data point. For example, within the Quality of life subindex, indicators such as happiness and freedom index scores will have a much more significant impact on the overall quality of life compared to indicators such as the amount of rainfall that falls annually or the proportion of the labour force in a country's army.
## 3) Calculation of final scores
Once a final score is calculated for each subindex, the four subindices are assigned their respective relative weightings and then combined to derive a final index score.

### Converting raw data to scores

In [1]:
import pandas as pd
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import inflect

To begin with, we import data from various data sources that are stored in separate Excel CSV files. These files are then merged together to create a single data frame that contains all the necessary data.

In [2]:
wb_data = pd.read_csv(r"raw_data/world_bank.csv", encoding = "ISO-8859-1")

In [3]:
wb_data.head()

,country,indicator,better_higher_lower,2000,2005,2010,2015,2019,2020,2021
0,Argentina,Access to electricity (% of population),higher,95.783287,96.863846,98.820000,99.625389,100.000000,100.0,NaN
1,Australia,Access to electricity (% of population),higher,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN
2,Austria,Access to electricity (% of population),higher,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN
3,Belgium,Access to electricity (% of population),higher,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN
4,Brazil,Access to electricity (% of population),higher,94.391716,97.093513,98.599998,99.710899,99.800003,100.0,NaN


The World Bank data provides metrics for the years 2000, 2005, 2010, 2015, 2019, 2020, and 2021. However, it's important to note that not all countries have data available for the most recent year (2021). Therefore, we need to identify the latest available year for each indicator and country to ensure accurate data representation.

In [4]:
def get_latest_value(row):
    years = ['2021', '2020', '2019', '2015', '2010', '2005', '2000']
    for year in years:
        if pd.notnull(row[year]):
            return row[year]
    return None 

In [5]:
wb_data['latest_value'] = wb_data.apply(get_latest_value, axis=1)

In [6]:
wb_data.head()

,country,indicator,better_higher_lower,2000,2005,2010,2015,2019,2020,2021,latest_value
0,Argentina,Access to electricity (% of population),higher,95.783287,96.863846,98.820000,99.625389,100.000000,100.0,NaN,100.0
1,Australia,Access to electricity (% of population),higher,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,100.0
2,Austria,Access to electricity (% of population),higher,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,100.0
3,Belgium,Access to electricity (% of population),higher,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,100.0
4,Brazil,Access to electricity (% of population),higher,94.391716,97.093513,98.599998,99.710899,99.800003,100.0,NaN,100.0


Removing yearly columns.

In [7]:
wb_data.drop(columns=['2000', '2005', '2010', '2015', '2019', '2020', '2021'], inplace=True)

In [8]:
wb_data.head()

,country,indicator,better_higher_lower,latest_value
0,Argentina,Access to electricity (% of population),higher,100.0
1,Australia,Access to electricity (% of population),higher,100.0
2,Austria,Access to electricity (% of population),higher,100.0
3,Belgium,Access to electricity (% of population),higher,100.0
4,Brazil,Access to electricity (% of population),higher,100.0


Combining other metrics from trading economics, numbeo and other sources.

In [9]:
te_cols = ['country', 'indicator', 'better_higher_lower', 'latest_value']
te_data = pd.read_csv(r"raw_data/trading_economics.csv", encoding = "ISO-8859-1", usecols=te_cols)

In [10]:
te_data.head()

,country,indicator,better_higher_lower,latest_value
0,Australia,Business Confidence,higher,0.0
1,Austria,Business Confidence,higher,-7.5
2,Belgium,Business Confidence,higher,-7.8
3,Brazil,Business Confidence,higher,48.8
4,Bulgaria,Business Confidence,higher,24.8


In [11]:
num_cols = ['country', 'indicator', 'better_higher_lower', 'latest_value']
num_data = pd.read_csv(r"raw_data/numbeo.csv", encoding = "ISO-8859-1", usecols=num_cols)

In [12]:
num_data.head()

,country,indicator,better_higher_lower,latest_value
0,Argentina,Rent cost index (points),lower,6.9
1,Australia,Rent cost index (points),lower,39.0
2,Austria,Rent cost index (points),lower,23.1
3,Belgium,Rent cost index (points),lower,22.4
4,Brazil,Rent cost index (points),lower,8.3


In [13]:
other_cols = ['country', 'indicator', 'better_higher_lower', 'latest_value']
other_cols = pd.read_csv(r"raw_data/other_sources.csv", encoding = "ISO-8859-1", usecols=other_cols)

In [14]:
other_cols.head()

,country,indicator,better_higher_lower,latest_value
0,Argentina,Vulnerability and exposure to natural hazards,lower,0.1561
1,Australia,Vulnerability and exposure to natural hazards,lower,0.2136
2,Austria,Vulnerability and exposure to natural hazards,lower,0.0114
3,Belgium,Vulnerability and exposure to natural hazards,lower,0.0416
4,Brazil,Vulnerability and exposure to natural hazards,lower,0.1215


In [15]:
all_data = (pd.concat([wb_data, te_data, num_data, other_cols])
            .reset_index(drop=True)
            .drop_duplicates(subset=['country', 'indicator']))

In [16]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              1968 non-null   object 
 1   indicator            1968 non-null   object 
 2   better_higher_lower  1968 non-null   object 
 3   latest_value         1933 non-null   float64
dtypes: float64(1), object(3)
memory usage: 61.6+ KB


We need to get the minimum and maximum country scores for each indicator for min max standardisation.

In [17]:
all_data['min'] = all_data.groupby('indicator')['latest_value'].transform('min')
all_data['max'] = all_data.groupby('indicator')['latest_value'].transform('max')

In [18]:
all_data.head()

,country,indicator,better_higher_lower,latest_value,min,max
0,Argentina,Access to electricity (% of population),higher,100.0,96.704636,100.0
1,Australia,Access to electricity (% of population),higher,100.0,96.704636,100.0
2,Austria,Access to electricity (% of population),higher,100.0,96.704636,100.0
3,Belgium,Access to electricity (% of population),higher,100.0,96.704636,100.0
4,Brazil,Access to electricity (% of population),higher,100.0,96.704636,100.0


Ranking countries by indicator based on if score is better higher or lower.

In [19]:
all_data.loc[all_data['better_higher_lower'] == 'higher', 'rank'] = (all_data
                                                                     .groupby('indicator', as_index=False)['latest_value']
                                                                     .rank(method='min', ascending=False))

In [20]:
all_data.loc[all_data['better_higher_lower'] == 'lower', 'rank'] = (all_data
                                                                    .groupby('indicator', as_index=False)['latest_value']
                                                                    .rank(method='min', ascending=True))

In [21]:
all_data.head()

,country,indicator,better_higher_lower,latest_value,min,max,rank
0,Argentina,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0
1,Australia,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0
2,Austria,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0
3,Belgium,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0
4,Brazil,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0


Standardise scores by indicator based on if score is better higher or lower.

In [22]:
all_data.loc[all_data['better_higher_lower'] == 'higher', 'standardized_score'] = (all_data['latest_value'] - all_data['min']) /  (all_data['max'] - all_data['min'])

In [23]:
all_data.loc[all_data['better_higher_lower'] == 'lower', 'standardized_score'] = 1 - (all_data['latest_value'] - all_data['min']) /  (all_data['max'] - all_data['min'])

In [24]:
all_data.head()

,country,indicator,better_higher_lower,latest_value,min,max,rank,standardized_score
0,Argentina,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0
1,Australia,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0
2,Austria,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0
3,Belgium,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0
4,Brazil,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0


In [25]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              1968 non-null   object 
 1   indicator            1968 non-null   object 
 2   better_higher_lower  1968 non-null   object 
 3   latest_value         1933 non-null   float64
 4   min                  1968 non-null   float64
 5   max                  1968 non-null   float64
 6   rank                 1933 non-null   float64
 7   standardized_score   1933 non-null   float64
dtypes: float64(5), object(3)
memory usage: 123.1+ KB


Fill in missing values with subindex mean score.

In [26]:
all_data['mean'] = all_data.groupby('indicator')['standardized_score'].transform('mean')

In [27]:
all_data.head()

,country,indicator,better_higher_lower,latest_value,min,max,rank,standardized_score,mean
0,Argentina,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0,0.944981
1,Australia,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0,0.944981
2,Austria,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0,0.944981
3,Belgium,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0,0.944981
4,Brazil,Access to electricity (% of population),higher,100.0,96.704636,100.0,1.0,1.0,0.944981


In [28]:
all_data.loc[all_data['standardized_score'].isna(), 'standardized_score'] = all_data['mean']

In [29]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              1968 non-null   object 
 1   indicator            1968 non-null   object 
 2   better_higher_lower  1968 non-null   object 
 3   latest_value         1933 non-null   float64
 4   min                  1968 non-null   float64
 5   max                  1968 non-null   float64
 6   rank                 1933 non-null   float64
 7   standardized_score   1968 non-null   float64
 8   mean                 1968 non-null   float64
dtypes: float64(6), object(3)
memory usage: 138.5+ KB


### Calculation of subindex scores

Add indicator id, subindex, area, total and subindex weights to indicators.

In [30]:
weights = pd.read_csv(r"weights.csv", encoding = "ISO-8859-1")

In [31]:
weights.head()

,id,indicator,subindex,area,subindex_weight,total_weight
0,1,GDP per Capita PPP,Economy,All,0.30,0.06
1,2,Unemployment Rate,Economy,All,0.20,0.04
2,3,Inflation Rate,Economy,All,0.15,0.03
3,4,Local purchasing power index (points),Economy,All,0.15,0.03
4,5,Average monthly wage,Economy,All,0.10,0.02


In [32]:
final_data = pd.merge(all_data, weights, on='indicator').drop(columns=['better_higher_lower', 'latest_value', 'min', 'max', 'mean'])

In [33]:
final_data.head()

,country,indicator,rank,standardized_score,id,subindex,area,subindex_weight,total_weight
0,Argentina,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01
1,Australia,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01
2,Austria,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01
3,Belgium,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01
4,Brazil,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01


In [34]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             1968 non-null   object 
 1   indicator           1968 non-null   object 
 2   rank                1933 non-null   float64
 3   standardized_score  1968 non-null   float64
 4   id                  1968 non-null   int64  
 5   subindex            1968 non-null   object 
 6   area                1968 non-null   object 
 7   subindex_weight     1968 non-null   float64
 8   total_weight        1968 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 138.5+ KB


Get subindex and total weighted scores.

In [35]:
final_data['subindex_score'] = final_data.standardized_score * final_data.subindex_weight

In [36]:
final_data['total_score'] = final_data.standardized_score * final_data.total_weight

In [37]:
final_data.head()

,country,indicator,rank,standardized_score,id,subindex,area,subindex_weight,total_weight,subindex_score,total_score
0,Argentina,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01,0.1,0.01
1,Australia,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01,0.1,0.01
2,Austria,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01,0.1,0.01
3,Belgium,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01,0.1,0.01
4,Brazil,Access to electricity (% of population),1.0,1.0,32,Quality of Life,Poverty and Equality,0.1,0.01,0.1,0.01


Check we have data for every indicator and all countries. There should be 48 indicators and 41 countries.

In [38]:
(final_data
 .sort_values(by='subindex_weight')
 .drop_duplicates('indicator')
 [['indicator', 'subindex_weight', 'total_score']].count())

indicator          48
subindex_weight    48
total_score        48
dtype: int64

In [39]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             1968 non-null   object 
 1   indicator           1968 non-null   object 
 2   rank                1933 non-null   float64
 3   standardized_score  1968 non-null   float64
 4   id                  1968 non-null   int64  
 5   subindex            1968 non-null   object 
 6   area                1968 non-null   object 
 7   subindex_weight     1968 non-null   float64
 8   total_weight        1968 non-null   float64
 9   subindex_score      1968 non-null   float64
 10  total_score         1968 non-null   float64
dtypes: float64(6), int64(1), object(4)
memory usage: 169.2+ KB


In [40]:
final_data.groupby(['indicator'])['country'].count()

indicator
Access to electricity (% of population)                                                   41
Armed forces personnel (% of total labor force)                                           41
Average monthly wage                                                                      41
Average precipitation in depth (mm per year)                                              41
Business Confidence                                                                       41
CEO health care index                                                                     41
CO2 emissions (metric tons per capita)                                                    41
Depression rate                                                                           41
Ease of doing business score (0 = lowest performance to 100 = best performance)           41
Economic freedom score                                                                    41
Exports of goods and services (annual % growth)             

In [41]:
list(final_data.subindex.unique())

['Quality of Life', 'Growth and Development', 'Economy', 'Cost of Living']

Getting economy subindex indicators and final score.

In [42]:
economy_final_ranking = (final_data[final_data.subindex=='Economy']
                         .groupby(['country'], as_index=False)['subindex_score'].sum()
                         .sort_values(by='subindex_score', ascending=False)
                         .reset_index(drop=True)
                         .assign(rank=lambda x: x.subindex_score.rank(ascending=False, method='min'),
                                 area='Final Economy Score'))

In [43]:
economy_final_ranking.reset_index(drop=True)

,country,subindex_score,rank,area
0,Switzerland,0.887440,1.0,Final Economy Score
1,Ireland,0.794682,2.0,Final Economy Score
2,United States,0.778177,3.0,Final Economy Score
3,Denmark,0.723698,4.0,Final Economy Score
4,Germany,0.704981,5.0,Final Economy Score
5,Norway,0.694834,6.0,Final Economy Score
6,Netherlands,0.675338,7.0,Final Economy Score
7,Canada,0.667504,8.0,Final Economy Score
8,Australia,0.658398,9.0,Final Economy Score
9,"Korea, Rep.",0.655711,10.0,Final Economy Score


To obtain the indicators and final score for the 'Quality of Life' subindex, we follow a different approach compared to other subindices. The 'Quality of Life' subindex is divided into four distinct areas, each with its own area weighting (0.2, 0.1, 0.3, 0.4). Consequently, we need to separate the quality of life indicators based on their respective areas, calculate the area scores, and subsequently combine them with their relative weightings to determine the final score.

In [44]:
list(final_data['area'].unique())

['Poverty and Equality',
 'Climate, Safety and Security',
 'All',
 'Education & Healthcare',
 'Happiness and Freedom']

In [45]:
poverty_final_ranking = (final_data[final_data['area']=='Poverty and Equality']
                         .groupby(['country'], as_index=False)['subindex_score'].sum()
                         .sort_values(by='subindex_score', ascending=False)
                         .reset_index(drop=True)
                         .assign(rank=lambda x: x.subindex_score.rank(ascending=False, method='min'),
                                 area='Poverty and Equality Score',
                                 weight=0.2)
                         .assign(weighted_subindex_score=lambda x: x.weight * x.subindex_score))

In [46]:
poverty_final_ranking.head()

,country,subindex_score,rank,area,weight,weighted_subindex_score
0,Belgium,0.974434,1.0,Poverty and Equality Score,0.2,0.194887
1,Denmark,0.955482,2.0,Poverty and Equality Score,0.2,0.191096
2,Norway,0.949844,3.0,Poverty and Equality Score,0.2,0.189969
3,Sweden,0.921186,4.0,Poverty and Equality Score,0.2,0.184237
4,Netherlands,0.917701,5.0,Poverty and Equality Score,0.2,0.183540


In [47]:
climate_final_ranking = (final_data[final_data['area']=='Climate, Safety and Security']
                         .groupby(['country'], as_index=False)['subindex_score'].sum()
                         .sort_values(by='subindex_score', ascending=False)
                         .reset_index(drop=True)
                         .assign(rank=lambda x: x.subindex_score.rank(ascending=False, method='min'),
                                 area='Climate, Safety and Security Score',
                                 weight=0.1)
                         .assign(weighted_subindex_score=lambda x: x.weight * x.subindex_score))

In [48]:
climate_final_ranking.head()

,country,subindex_score,rank,area,weight,weighted_subindex_score
0,Israel,0.864160,1.0,"Climate, Safety and Security Score",0.1,0.086416
1,Sweden,0.835015,2.0,"Climate, Safety and Security Score",0.1,0.083502
2,Lithuania,0.819215,3.0,"Climate, Safety and Security Score",0.1,0.081921
3,Denmark,0.818862,4.0,"Climate, Safety and Security Score",0.1,0.081886
4,Switzerland,0.810850,5.0,"Climate, Safety and Security Score",0.1,0.081085


In [49]:
education_final_ranking = (final_data[final_data['area']=='Education & Healthcare']
                           .groupby(['country'], as_index=False)['subindex_score'].sum()
                           .sort_values(by='subindex_score', ascending=False)
                           .reset_index(drop=True)
                           .assign(rank=lambda x: x.subindex_score.rank(ascending=False, method='min'),
                                 area='Education & Healthcare Score',
                                 weight=0.3)
                           .assign(weighted_subindex_score=lambda x: x.weight * x.subindex_score))

In [50]:
education_final_ranking.head()

,country,subindex_score,rank,area,weight,weighted_subindex_score
0,Denmark,0.837026,1.0,Education & Healthcare Score,0.3,0.251108
1,Netherlands,0.759991,2.0,Education & Healthcare Score,0.3,0.227997
2,Norway,0.738555,3.0,Education & Healthcare Score,0.3,0.221566
3,Finland,0.731824,4.0,Education & Healthcare Score,0.3,0.219547
4,Japan,0.725710,5.0,Education & Healthcare Score,0.3,0.217713


In [51]:
happiness_final_ranking = (final_data[final_data['area']=='Happiness and Freedom']
                           .groupby(['country'], as_index=False)['subindex_score'].sum()
                           .sort_values(by='subindex_score', ascending=False)
                           .reset_index(drop=True)
                           .assign(rank=lambda x: x.subindex_score.rank(ascending=False, method='min'),
                                 area='Happiness and Freedom Score',
                                 weight=0.4)
                           .assign(weighted_subindex_score=lambda x: x.weight * x.subindex_score))

In [52]:
happiness_final_ranking.head()

,country,subindex_score,rank,area,weight,weighted_subindex_score
0,New Zealand,0.832955,1.0,Happiness and Freedom Score,0.4,0.333182
1,Denmark,0.784889,2.0,Happiness and Freedom Score,0.4,0.313956
2,Canada,0.764620,3.0,Happiness and Freedom Score,0.4,0.305848
3,Australia,0.751050,4.0,Happiness and Freedom Score,0.4,0.300420
4,Switzerland,0.750781,5.0,Happiness and Freedom Score,0.4,0.300312


Concatinating all 4 quality of life scores.

In [53]:
final_quality_of_life = pd.concat([poverty_final_ranking, climate_final_ranking, education_final_ranking,happiness_final_ranking])

In [54]:
final_quality_of_life.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 0 to 40
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  164 non-null    object 
 1   subindex_score           164 non-null    float64
 2   rank                     164 non-null    float64
 3   area                     164 non-null    object 
 4   weight                   164 non-null    float64
 5   weighted_subindex_score  164 non-null    float64
dtypes: float64(4), object(2)
memory usage: 9.0+ KB


Getting total quality of life score.

In [55]:
final_qual_score = (final_quality_of_life
                    .groupby('country', as_index=False)['weighted_subindex_score'].sum()
                    .assign(rank=lambda x: x.weighted_subindex_score.rank(ascending=False, method='min'),
                            area='Final Quality of Life Score'))

In [56]:
final_qual_score.head()

,country,weighted_subindex_score,rank,area
0,Argentina,0.469029,37.0,Final Quality of Life Score
1,Australia,0.737815,9.0,Final Quality of Life Score
2,Austria,0.694426,13.0,Final Quality of Life Score
3,Belgium,0.737908,8.0,Final Quality of Life Score
4,Brazil,0.345392,40.0,Final Quality of Life Score


Adding final score to area scores.

In [57]:
final_quality_of_life = (pd.concat([final_quality_of_life.drop(columns=['weight', 'weighted_subindex_score']),
                                    final_qual_score.rename(columns={'weighted_subindex_score': 'subindex_score'})]))

In [58]:
final_quality_of_life.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205 entries, 0 to 40
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         205 non-null    object 
 1   subindex_score  205 non-null    float64
 2   rank            205 non-null    float64
 3   area            205 non-null    object 
dtypes: float64(2), object(2)
memory usage: 8.0+ KB


In [59]:
final_quality_of_life.reset_index(drop=True)

,country,subindex_score,rank,area
0,Belgium,0.974434,1.0,Poverty and Equality Score
1,Denmark,0.955482,2.0,Poverty and Equality Score
2,Norway,0.949844,3.0,Poverty and Equality Score
3,Sweden,0.921186,4.0,Poverty and Equality Score
4,Netherlands,0.917701,5.0,Poverty and Equality Score
5,Germany,0.910880,6.0,Poverty and Equality Score
6,Spain,0.908488,7.0,Poverty and Equality Score
7,Finland,0.904478,8.0,Poverty and Equality Score
8,France,0.902361,9.0,Poverty and Equality Score
9,New Zealand,0.892273,10.0,Poverty and Equality Score


Getting cost of living subindex indicators and final score.

In [60]:
cost_of_living_final_ranking = (final_data[final_data['subindex']=='Cost of Living']
                                .groupby(['country'], as_index=False)['subindex_score'].sum()
                                .sort_values(by='subindex_score', ascending=False)
                                .reset_index(drop=True)
                                .assign(rank=lambda x: x.subindex_score.rank(ascending=False, method='min'),
                                        area='Final Cost of living Score'))

In [61]:
cost_of_living_final_ranking

,country,subindex_score,rank,area
0,Bulgaria,0.822577,1.0,Final Cost of living Score
1,Romania,0.768423,2.0,Final Cost of living Score
2,Estonia,0.684274,3.0,Final Cost of living Score
3,Hungary,0.674833,4.0,Final Cost of living Score
4,Lithuania,0.666848,5.0,Final Cost of living Score
5,Costa Rica,0.650979,6.0,Final Cost of living Score
6,Panama,0.646243,7.0,Final Cost of living Score
7,Tunisia,0.642487,8.0,Final Cost of living Score
8,Croatia,0.625851,9.0,Final Cost of living Score
9,Belgium,0.615161,10.0,Final Cost of living Score


Getting growth and development subindex indicators and final score.

In [62]:
growth_final_ranking = (final_data[final_data['subindex']=='Growth and Development']
                        .groupby(['country'], as_index=False)['subindex_score'].sum()
                        .sort_values(by='subindex_score', ascending=False)
                        .reset_index(drop=True)
                        .assign(rank=lambda x: x.subindex_score.rank(ascending=False, method='min'),
                                area='Final Growth and Development Score'))

In [63]:
growth_final_ranking

,country,subindex_score,rank,area
0,Ireland,0.621137,1.0,Final Growth and Development Score
1,United States,0.595113,2.0,Final Growth and Development Score
2,France,0.564281,3.0,Final Growth and Development Score
3,Poland,0.549216,4.0,Final Growth and Development Score
4,Spain,0.523135,5.0,Final Growth and Development Score
5,Denmark,0.514168,6.0,Final Growth and Development Score
6,New Zealand,0.511081,7.0,Final Growth and Development Score
7,United Kingdom,0.509855,8.0,Final Growth and Development Score
8,Canada,0.499658,9.0,Final Growth and Development Score
9,Lithuania,0.495650,10.0,Final Growth and Development Score


### Calculation of final scores

The final index score is obtained by summing the weighted scores of all the indicators.

In [64]:
final_scores = (final_data
                .groupby('country', as_index=False)['total_score'].sum()
                .assign(rank=lambda x: x.total_score.rank(ascending=False, method='min'),
                        area='Final Score')
                .rename(columns={'total_score': 'subindex_score'}))

In [65]:
final_scores.sort_values(by='rank').reset_index(drop=True)

,country,subindex_score,rank,area
0,Denmark,0.719658,1.0,Final Score
1,Norway,0.674120,2.0,Final Score
2,Ireland,0.662085,3.0,Final Score
3,Switzerland,0.661969,4.0,Final Score
4,Sweden,0.658507,5.0,Final Score
5,Germany,0.651510,6.0,Final Score
6,Netherlands,0.651481,7.0,Final Score
7,New Zealand,0.651373,8.0,Final Score
8,United States,0.648600,9.0,Final Score
9,Belgium,0.647564,10.0,Final Score


Concatinating all scores into single df.

In [66]:
scores = pd.concat([final_scores,
                    economy_final_ranking,
                    final_quality_of_life,
                    cost_of_living_final_ranking,
                    growth_final_ranking])

In [67]:
scores.sort_values(by='country').head(7)

,country,subindex_score,rank,area
0,Argentina,0.408601,38.0,Final Score
39,Argentina,0.246261,40.0,Final Economy Score
35,Argentina,0.679279,36.0,Poverty and Equality Score
37,Argentina,0.279758,38.0,Final Growth and Development Score
29,Argentina,0.623053,30.0,"Climate, Safety and Security Score"
27,Argentina,0.410825,28.0,Education & Healthcare Score
35,Argentina,0.482538,36.0,Final Cost of living Score


Adding scores to final_data df.

In [68]:
final_df = (pd.concat([scores, final_data[['indicator', 'area', 'country', 'rank', 'standardized_score', 'id']]
                       .rename(columns={'standardized_score': 'subindex_score'})]))

In [69]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2337 entries, 0 to 1967
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         2337 non-null   object 
 1   subindex_score  2337 non-null   float64
 2   rank            2302 non-null   float64
 3   area            2337 non-null   object 
 4   indicator       1968 non-null   object 
 5   id              1968 non-null   float64
dtypes: float64(3), object(3)
memory usage: 127.8+ KB


Adding missing ID's to new scores.

In [70]:
final_df[final_df['country'] == 'Argentina'].sort_values(by='id')

,country,subindex_score,rank,area,indicator,id
1066,Argentina,0.120839,36.0,All,GDP per Capita PPP,1.0
1230,Argentina,0.674242,25.0,All,Unemployment Rate,2.0
1148,Argentina,0.000000,41.0,All,Inflation Rate,3.0
1435,Argentina,0.101974,36.0,All,Local purchasing power index (points),4.0
1300,Argentina,0.122890,30.0,All,Average monthly wage,5.0
1020,Argentina,0.475758,NaN,All,Business Confidence,6.0
820,Argentina,0.000000,36.0,Happiness and Freedom,Strength of legal rights index (0=weak to 12=s...,7.0
1640,Argentina,0.695793,22.0,Happiness and Freedom,Personal freedom score,8.0
1914,Argentina,0.491233,29.0,Happiness and Freedom,Happiness index,9.0
1681,Argentina,0.000000,41.0,Happiness and Freedom,Economic freedom score,10.0


In [71]:
final_df.loc[final_df['area'] == 'Final Economy Score', 'id'] = 0.5
final_df.loc[final_df['area'] == 'Poverty and Equality Score', 'id'] = 28.5
final_df.loc[final_df['area'] == 'Climate, Safety and Security Score', 'id'] = 22.5
final_df.loc[final_df['area'] == 'Education & Healthcare Score', 'id'] = 12.5
final_df.loc[final_df['area'] == 'Happiness and Freedom Score', 'id'] = 6.5
final_df.loc[final_df['area'] == 'Final Cost of living Score', 'id'] = 36.5
final_df.loc[final_df['area'] == 'Final Growth and Development Score', 'id'] = 42.5

In [72]:
final_df[final_df['country'] == 'Argentina'].sort_values(by='id')

,country,subindex_score,rank,area,indicator,id
39,Argentina,0.246261,40.0,Final Economy Score,NaN,0.5
1066,Argentina,0.120839,36.0,All,GDP per Capita PPP,1.0
1230,Argentina,0.674242,25.0,All,Unemployment Rate,2.0
1148,Argentina,0.000000,41.0,All,Inflation Rate,3.0
1435,Argentina,0.101974,36.0,All,Local purchasing power index (points),4.0
1300,Argentina,0.122890,30.0,All,Average monthly wage,5.0
1020,Argentina,0.475758,NaN,All,Business Confidence,6.0
37,Argentina,0.369050,38.0,Happiness and Freedom Score,NaN,6.5
820,Argentina,0.000000,36.0,Happiness and Freedom,Strength of legal rights index (0=weak to 12=s...,7.0
1640,Argentina,0.695793,22.0,Happiness and Freedom,Personal freedom score,8.0


Adding suffixes to the ranks, indicators that have no available data will be marked with a dash ('-') to indicate the absence of data and rank. However, these indicators will still be assigned the mean score of the respective indicator to ensure continuity when compared with other countries.

In [73]:
def add_suffix(number):
    p = inflect.engine()
    number = str(number).replace('.0', '')
    try:
        suffix = p.ordinal(number)
    except:
        suffix = '-'
    return suffix

In [74]:
final_df['rank'] = final_df['rank'].apply(add_suffix).astype('str').replace('nanth','-')

In [75]:
final_df.head()

,country,subindex_score,rank,area,indicator,id
0,Argentina,0.408601,38th,Final Score,NaN,NaN
1,Australia,0.640162,13th,Final Score,NaN,NaN
2,Austria,0.605712,18th,Final Score,NaN,NaN
3,Belgium,0.647564,10th,Final Score,NaN,NaN
4,Brazil,0.359908,40th,Final Score,NaN,NaN


In [76]:
final_df[final_df['area'] == 'Final Score']['subindex_score'].mean()

0.56457824376777

In [77]:
final_df[final_df['area'] == 'Final Economy Score']['subindex_score'].mean()

0.5236367138821586

In [78]:
final_df[final_df['area'] == 'Final Quality of Life Score']['subindex_score'].mean()

0.622877262744689

In [79]:
final_df[final_df['area'] == 'Final Cost of living Score']['subindex_score'].mean()

0.5553419216981211

In [80]:
final_df[final_df['area'] == 'Final Growth and Development Score']['subindex_score'].mean()

0.4113879212093164

Getting all final inidicator, sub index and total scores for each country stored in individual files.

In [81]:
countries = list(final_df['country'].unique())

In [82]:
countries

['Argentina',
 'Australia',
 'Austria',
 'Belgium',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'Costa Rica',
 'Croatia',
 'Czechia',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Indonesia',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Korea, Rep.',
 'Lithuania',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Panama',
 'Peru',
 'Poland',
 'Portugal',
 'Romania',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Tunisia',
 'United Kingdom',
 'United States',
 'Uruguay']

In [83]:
for c in countries:
    df = final_df[final_df['country'] == c].sort_values(by='id')
    df.to_csv(f'countries/{str(c)}.csv', index=False)